In [1]:
#pip install beautifulsoup4
import bs4
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen
import requests
from neo4j import GraphDatabase

In [2]:
global cont
cont = 0
#CLASE PARA CREAR NODO CANDIDATO
class HelloWorldExample(object): 
    def __init__(self):
        self._driver = GraphDatabase.driver("bolt:neo4j://localhost:7687", auth=("neo4j", "neo4jj"), encrypted=False)
     
    def close(self):
        self._driver.close()

    def print_greeting(self, message):
        with self._driver.session() as session:
            greeting = session.write_transaction(self._create_and_return_greeting, message)
            print(greeting)  
            
    def print_create_new(self, message, titulo, descripcion, nombre, link):
        with self._driver.session() as session:
            greeting = session.write_transaction(self._create_and_return_news, message, titulo, descripcion, nombre, link)
            print(greeting)  
            
    def relation(self, nombre):
        with self._driver.session() as session:
            greeting = session.write_transaction(self._create_and_return_relation, nombre)
            print(greeting) 
            
    def relation2(self, nombre):
        with self._driver.session() as session:
            greeting = session.write_transaction(self._create_and_return_relation2, nombre)
            print(greeting)  
    
    def relation3(self, nombre):
        with self._driver.session() as session:
            greeting = session.write_transaction(self._create_and_return_relation3, nombre)
            print(greeting)

    @staticmethod
    def _create_and_return_greeting(tx, message):
        result = tx.run("CREATE (Lloret: Candidato {nombres:'Juan Lloret',movimiento: 'Centro Democrático',cargo: 'Asambleista Provincial'}) "
                        "SET Lloret.message = $message "
                        "RETURN Lloret.message + ', from node ' + id(Lloret)", message=message)
        result2 = tx.run("CREATE (Correa: Candidato {nombres:'Pierina Correa', movimiento: 'Centro Democrático', cargo: 'Asambleista Nacional'}) "
                        "SET Correa.message = $message "
                        "RETURN Correa.message + ', from node ' + id(Correa)", message=message)
        result3 = tx.run("CREATE (Arauz: Candidato {nombres:'Andres Arauz', movimiento: 'Centro Democrático', cargo: 'Presidente'}) "
                        "SET Arauz.message = $message "
                        "RETURN Arauz.message + ', from node ' + id(Arauz)", message=message)
        result4 = tx.run("match(Arauz:Candidato {nombres:'Andres Arauz'})  match(Correa:Candidato {nombres:'Pierina Correa'}) match(Lloret:Candidato {nombres:'Juan Lloret'}) create (Arauz)-[:ES_DEL_MISMO_PARTIDO]->(Correa), (Arauz)-[:ES_DEL_MISMO_PARTIDO]->(Lloret)")
        return result.single()[0], result2.single()[0], result3.single()[0]
    
    @staticmethod
    def _create_and_return_news(tx, message, titulo, descripcion, nombre, link):
        result = tx.run("CREATE ("+nombre+":Noticias {nombres:'"+nombre+"',titulo:'"+str(titulo)+"',descripcion:'"+str(descripcion)+"',link:'"+str(link)+"'}) "
                        "SET "+nombre+".message = $message "
                        "RETURN "+nombre+".message + ', from node ' + id("+nombre+")", message=message)
        return result.single()[0]
        
    @staticmethod
    def _create_and_return_relation(tx, nombre):
        result22 = tx.run("match("+nombre+":Noticias {nombres:'"+nombre+"'}) match(Lloret:Candidato {nombres:'Juan Lloret'}) create("+nombre+")-[:NOTICIA_DE]->(Lloret)")
    
    @staticmethod
    def _create_and_return_relation2(tx, nombre):
        result11 = tx.run("match("+nombre+":Noticias {nombres:'"+nombre+"'}) match(Correa:Candidato {nombres:'Pierina Correa'}) create("+nombre+")-[:NOTICIA_DE]->(Correa)")
    
    @staticmethod
    def _create_and_return_relation3(tx, nombre):
        result33 = tx.run("match("+nombre+":Noticias {nombres:'"+nombre+"'}) match(Arauz:Candidato {nombres:'Andres Arauz'}) create("+nombre+")-[:NOTICIA_DE]->(Arauz)")
        
grafo = HelloWorldExample()
grafo.print_greeting("Se crea el nodo candidato")

('Se crea el nodo candidato, from node 0', 'Se crea el nodo candidato, from node 1', 'Se crea el nodo candidato, from node 2')


In [3]:
url_general = "https://www.google.com/"
url = "https://www.google.com/search?q=juan+crist%C3%B3bal+lloret&hl=es&sxsrf=ALeKk003iUbQ29XU00-Hmiwu9_0t_zx-OA:1607487353447&source=lnms&tbm=nws&sa=X&ved=2ahUKEwjvlruFhcDtAhVm0FkKHQIzCX4Q_AUoA3oECAQQBQ&biw=768&bih=724"
#url = "https://www.bing.com/search?q=Arauz&form=QBLH&sp=-1&pq=arauz&sc=8-5&qs=n&sk=&cvid=C87939D5C9C24D5C81A8DD0A885C6899"

def news(url): 
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    html_doc = urlopen(req).read()
    with requests.Session() as c:
        soupp = soup(html_doc, 'html5lib')

        #print(soupp.prettify())
        for item in soupp.find_all('div', 'ZINbbc xpd O9g5cc uUPGi'):
            titulo = (item.find("div", attrs={'class': 'BNeawe vvjwJb AP7Wnd'}).get_text())
            descripcion = (item.find("div", attrs={'class': 'BNeawe s3v9rd AP7Wnd'}).get_text())
            raw_link = (item.find('a', href=True)['href'])
            link = (raw_link.split("/url?q=")[1]).split('&sa=U&')[0]
            titulo = str(titulo).replace(",","").replace("...","").replace("'","").replace("\"","")
            descripcion = str(descripcion).replace(",","").replace("...","").replace("'","").replace("\"","")
            descripcion = descripcion.split(" · ")[1]
            global cont
            cont = cont + 1
            nombre = "Noticia" + str(cont)
            grafo.print_create_new("Se crea el nodo noticia", titulo, descripcion, nombre, link)
            grafo.relation(nombre)
        next = soupp.find('a',attrs={'aria-label': 'Página siguiente'})
        next = (next['href'])
        url = url_general + next

        news(url)  
try:
    news(url)
except(TypeError):
    print("Ya no encuentra mas resultados")

Se crea el nodo noticia, from node 3
None
Se crea el nodo noticia, from node 4
None
Se crea el nodo noticia, from node 5
None
Se crea el nodo noticia, from node 6
None
Se crea el nodo noticia, from node 7
None
Se crea el nodo noticia, from node 8
None
Se crea el nodo noticia, from node 9
None
Se crea el nodo noticia, from node 10
None
Se crea el nodo noticia, from node 11
None
Se crea el nodo noticia, from node 12
None
Se crea el nodo noticia, from node 13
None
Se crea el nodo noticia, from node 14
None
Se crea el nodo noticia, from node 15
None
Se crea el nodo noticia, from node 16
None
Se crea el nodo noticia, from node 17
None
Se crea el nodo noticia, from node 18
None
Se crea el nodo noticia, from node 19
None
Se crea el nodo noticia, from node 20
None
Se crea el nodo noticia, from node 21
None
Se crea el nodo noticia, from node 22
None
Se crea el nodo noticia, from node 23
None
Se crea el nodo noticia, from node 24
None
Se crea el nodo noticia, from node 25
None
Se crea el nodo no

Se crea el nodo noticia, from node 193
None
Se crea el nodo noticia, from node 194
None
Se crea el nodo noticia, from node 195
None
Se crea el nodo noticia, from node 196
None
Se crea el nodo noticia, from node 197
None
Se crea el nodo noticia, from node 198
None
Se crea el nodo noticia, from node 199
None
Se crea el nodo noticia, from node 200
None
Se crea el nodo noticia, from node 201
None
Se crea el nodo noticia, from node 202
None
Se crea el nodo noticia, from node 203
None
Se crea el nodo noticia, from node 204
None
Se crea el nodo noticia, from node 205
None
Se crea el nodo noticia, from node 206
None
Se crea el nodo noticia, from node 207
None
Se crea el nodo noticia, from node 208
None
Se crea el nodo noticia, from node 209
None
Se crea el nodo noticia, from node 210
None
Se crea el nodo noticia, from node 211
None
Se crea el nodo noticia, from node 212
None
Se crea el nodo noticia, from node 213
None
Se crea el nodo noticia, from node 214
None
Se crea el nodo noticia, from no

In [4]:
url_general = "https://www.google.com/"
url2 = "https://www.google.com/search?biw=767&bih=716&tbm=nws&sxsrf=ALeKk02DfdEZT3mUleo3sDcFP5PbcmTSnw%3A1608496681371&ei=KbbfX-SNFsqD5wK-vKGIDA&q=pierina+correa+delgado&oq=pierina+correa+delgado&gs_l=psy-ab.3...0.0.0.9793056.0.0.0.0.0.0.0.0..0.0....0...1c..64.psy-ab..0.0.0....0.gPth6DeEzZw"

def news(url2): 
    req = Request(url2, headers={'User-Agent': 'Mozilla/5.0'})
    html_doc = urlopen(req).read()
    with requests.Session() as c:
        soupp = soup(html_doc, 'html5lib')

        #print(soupp.prettify())
        for item in soupp.find_all('div', 'ZINbbc xpd O9g5cc uUPGi'):
            titulo = (item.find("div", attrs={'class': 'BNeawe vvjwJb AP7Wnd'}).get_text())
            descripcion = (item.find("div", attrs={'class': 'BNeawe s3v9rd AP7Wnd'}).get_text())
            raw_link = (item.find('a', href=True)['href'])
            link = (raw_link.split("/url?q=")[1]).split('&sa=U&')[0]
            titulo = str(titulo).replace(",","").replace("...","").replace("'","").replace("\"","")
            descripcion = str(descripcion).replace(",","").replace("...","").replace("'","").replace("\"","")
            descripcion = descripcion.split(" · ")[1]
            global cont
            cont = cont + 1
            nombre = "Noticia" + str(cont)
            grafo.print_create_new("Se crea el nodo noticia", titulo, descripcion, nombre, link)
            grafo.relation2(nombre)
        next = soupp.find('a',attrs={'aria-label': 'Página siguiente'})
        next = (next['href'])
        url2 = url_general + next
        news(url2)

try:
    news(url2)
except(TypeError):
    print("Ya no encuentra mas resultados")

Se crea el nodo noticia, from node 263
None
Se crea el nodo noticia, from node 264
None
Se crea el nodo noticia, from node 265
None
Se crea el nodo noticia, from node 266
None
Se crea el nodo noticia, from node 267
None
Se crea el nodo noticia, from node 268
None
Se crea el nodo noticia, from node 269
None
Se crea el nodo noticia, from node 270
None
Se crea el nodo noticia, from node 271
None
Se crea el nodo noticia, from node 272
None
Se crea el nodo noticia, from node 273
None
Se crea el nodo noticia, from node 274
None
Se crea el nodo noticia, from node 275
None
Se crea el nodo noticia, from node 276
None
Se crea el nodo noticia, from node 277
None
Se crea el nodo noticia, from node 278
None
Se crea el nodo noticia, from node 279
None
Se crea el nodo noticia, from node 280
None
Se crea el nodo noticia, from node 281
None
Se crea el nodo noticia, from node 282
None
Se crea el nodo noticia, from node 283
None
Se crea el nodo noticia, from node 284
None
Se crea el nodo noticia, from no

In [5]:
url_general = "https://www.google.com/"
url3 = "https://www.google.com/search?q=andres+arauz&sxsrf=ALeKk02M-81xhKOA2nAgkb-wiLC_-WuMkw:1608496564400&source=lnms&tbm=nws&sa=X&ved=2ahUKEwiIlt_StN3tAhVBq1kKHXsVB_IQ_AUoA3oECCIQBQ&cshid=1608496720416275&biw=767&bih=716"

def news(url3): 
    req = Request(url3, headers={'User-Agent': 'Mozilla/5.0'})
    html_doc = urlopen(req).read()
    with requests.Session() as c:
        soupp = soup(html_doc, 'html5lib')

        #print(soupp.prettify())
        for item in soupp.find_all('div', 'ZINbbc xpd O9g5cc uUPGi'):
            titulo = (item.find("div", attrs={'class': 'BNeawe vvjwJb AP7Wnd'}).get_text())
            descripcion = (item.find("div", attrs={'class': 'BNeawe s3v9rd AP7Wnd'}).get_text())
            raw_link = (item.find('a', href=True)['href'])
            link = (raw_link.split("/url?q=")[1]).split('&sa=U&')[0]
            titulo = str(titulo).replace(",","").replace("...","").replace("'","").replace("\"","")
            descripcion = str(descripcion).replace(",","").replace("...","").replace("'","").replace("\"","")
            descripcion = descripcion.split(" · ")[1]
            global cont
            cont = cont + 1
            nombre = "Noticia" + str(cont)
            grafo.print_create_new("Se crea el nodo noticia", titulo, descripcion, nombre, link)
            grafo.relation3(nombre)
        next = soupp.find('a',attrs={'aria-label': 'Página siguiente'})
        next = (next['href'])
        url3 = url_general + next
        news(url3)
try:
    news(url3)
except(TypeError):
    print("Ya no encuentra mas reusltados")

Se crea el nodo noticia, from node 428
None
Se crea el nodo noticia, from node 429
None
Se crea el nodo noticia, from node 430
None
Se crea el nodo noticia, from node 431
None
Se crea el nodo noticia, from node 432
None
Se crea el nodo noticia, from node 433
None
Se crea el nodo noticia, from node 434
None
Se crea el nodo noticia, from node 435
None
Se crea el nodo noticia, from node 436
None
Se crea el nodo noticia, from node 437
None
Se crea el nodo noticia, from node 438
None
Se crea el nodo noticia, from node 439
None
Se crea el nodo noticia, from node 440
None
Se crea el nodo noticia, from node 441
None
Se crea el nodo noticia, from node 442
None
Se crea el nodo noticia, from node 443
None
Se crea el nodo noticia, from node 444
None
Se crea el nodo noticia, from node 445
None
Se crea el nodo noticia, from node 446
None
Se crea el nodo noticia, from node 447
None
Se crea el nodo noticia, from node 448
None
Se crea el nodo noticia, from node 449
None
Se crea el nodo noticia, from no